In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
import warnings
warnings.filterwarnings("ignore")

In [2]:
from tqdm import tqdm

In [3]:
# Read data
data = pd.read_csv('data.csv')
data['month'] = data['month'].apply(lambda x: datetime.strptime(x, '%Y-%m')) #assume each date per month is the last day
data.set_index(['month'],inplace=True)
# No missing month

### Functions for Grid Search

- For each grid, take mean over the all possible paths (119 MC)
- Grid Search - Three dimensions (10 * 10 * 10) 

In [4]:
from PathGen import *

def Experiments(MC,data,CAP,SAVE,g,ratio,years=[60,84],tax=0):
    # MC simulation --------------------------------------------------------------------------------------------
    # Years need to be 2 examples
    Simu_table = pd.DataFrame(index = range(MC),\
    columns = ['5Y_cap','7Y_cap','5Y_sharpe','7Y_sharpe'])

    for mc in range(MC):
        temdf = MC_generate(mc,data,CAP,SAVE,g,ratio)
        temdf.reset_index(inplace=True,drop=True)
        # Loc out end of different years
        rst = temdf.loc[np.r_[years[0]-1,years[1]-1],'cap_total']
        rst2 = temdf.loc[np.r_[years[0]-1,years[1]-1],'cap_input'] 
        # Total Capital after tax for different dates
        Simu_table.iloc[mc,:2] = rst - tax*(rst - rst2) # Tax Adjustment (only pay for capital gain)
        Simu_table.iloc[mc,2] = ret_annual_sharpe(temdf.iloc[:years[0],1])
        Simu_table.iloc[mc,3] = ret_annual_sharpe(temdf.iloc[:years[1],1])
        
    return Simu_table

In [21]:
# Grid Generation here --------------------------------------------------------------------

# Allow shorting
# Generate a DataFrame with 1000 rows and 4 columns
# Set a random seed for reproducibility
np.random.seed(535)

# Generate a DataFrame with 1000 rows and 4 columns
rand_data = np.random.uniform(-1, 2, size=(1400, 3))
df_grid = pd.DataFrame(rand_data, columns=['x1', 'x2', 'x3'])

# Calculate x4 as 1 minus the sum of x1 and x2
df_grid['x4'] = 1 - df_grid['x1'] - df_grid['x2']

# Drop rows based on the condition
condition = (df_grid['x4'] >= -1) & (df_grid['x4'] <= 2)
df_grid = df_grid[condition]

df_grid.reset_index(drop=True,inplace=True)
df_grid = df_grid.iloc[:500,:]

# Display the sorted DataFrame
print(df_grid)

           x1        x2        x3        x4
0    0.086479  1.240959 -0.509618 -0.327439
1    0.028712  0.443163  1.443732  0.528125
2    0.599065  0.659452 -0.474361 -0.258517
3    0.274101  1.063064  1.837332 -0.337165
4    0.639430 -0.684515  1.923394  1.045084
..        ...       ...       ...       ...
495  0.604734 -0.398068  1.074430  0.793334
496 -0.093313 -0.722981 -0.196339  1.816294
497  0.511702  0.402142 -0.981169  0.086156
498  1.527411 -0.971157  1.537272  0.443746
499 -0.516297  1.157367  1.019360  0.358930

[500 rows x 4 columns]


In [5]:
# Does not allow shorting
# Generate a DataFrame with 1000 rows and 4 columns
np.random.seed(535)

rand_data = np.random.uniform(0, 2, size=(9000, 3))
df_grid2 = pd.DataFrame(rand_data, columns=['x1', 'x2', 'x3'])

# Calculate x4 as 1 minus the sum of x1 and x2
df_grid2['x4'] = 1 - df_grid2['x1'] - df_grid2['x2']

# Drop rows based on the condition
condition = (df_grid2['x4'] >= 0) & (df_grid2['x4'] <= 2)
df_grid2 = df_grid2[condition]

df_grid2.reset_index(drop=True,inplace=True)
df_grid2 = df_grid2.iloc[:500,:]

# Display the sorted DataFrame
print(df_grid2)

           x1        x2        x3        x4
0    0.252218  0.389102  0.879787  0.358679
1    0.502337  0.319383  0.845133  0.178280
2    0.202873  0.378159  0.650015  0.418968
3    0.325748  0.632399  1.387910  0.041853
4    0.111440  0.710112  1.133961  0.178449
..        ...       ...       ...       ...
495  0.111281  0.331458  0.091367  0.557261
496  0.477581  0.387882  0.342385  0.134537
497  0.818098  0.125310  0.095685  0.056592
498  0.297952  0.162477  1.990411  0.539571
499  0.418443  0.154279  0.905400  0.427278

[500 rows x 4 columns]


In [8]:
df_grid = df_grid2

In [13]:
data2 = data.copy()
data2.iloc[:,-1] = data2.iloc[:,-1]*20

In [16]:
data2

,SPY,TLT,BAB,rf
month,,,,
2002-08-01,0.006500,0.052090,0.0314,0.028
2002-09-01,-0.108421,0.044243,0.0552,0.028
2002-10-01,0.087761,-0.037392,-0.0886,0.028
2002-11-01,0.058490,-0.008238,-0.1444,0.024
2002-12-01,-0.058631,0.043301,0.1231,0.022
...,...,...,...,...
2022-02-01,-0.029962,-0.016365,-0.0206,0.000
2022-03-01,0.036962,-0.054305,0.0199,0.002
2022-04-01,-0.087209,-0.094596,0.0157,0.002


In [17]:
# search here
# Second run, Change data ---------------------------------------------------------------------
# data2 = data.copy()
# data2.iloc[:,-1] = data2.iloc[:,-1]*20

SAVE = 40000
CAP = 50000
g = 0.05

MC = 119
df_compare_5Y = pd.DataFrame(index=range(500),columns = ['Sharpe','Utility','Prob'])
df_compare_7Y = pd.DataFrame(index=range(500),columns = ['Sharpe','Utility','Prob'])

for idx in tqdm(range(500)):
    ratio = (df_grid.iloc[idx,:]).to_list()
    temp_df = Experiments(MC,data2,CAP,SAVE,g,ratio,years=[60,84],tax=0)
    uiti_5Y = temp_df['5Y_cap'].apply(lambda x: np.log(x))
    uiti_7Y = temp_df['7Y_cap'].apply(lambda x: np.log(x))
    df_compare_5Y.iloc[idx,0] = np.nanmean(temp_df['5Y_sharpe'])
    df_compare_7Y.iloc[idx,0] = np.nanmean(temp_df['7Y_sharpe'])
    df_compare_5Y.iloc[idx,1] = np.nanmean(uiti_5Y)
    df_compare_7Y.iloc[idx,1] = np.nanmean(uiti_7Y)
    df_compare_5Y.iloc[idx,2] = (temp_df['5Y_cap'] >= 500000).mean()
    df_compare_7Y.iloc[idx,2] = (temp_df['7Y_cap'] >= 500000).mean()

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [10:51<00:00,  1.30s/it]


In [19]:
df_compare_5Y['Sharpe'] = pd.to_numeric(df_compare_5Y['Sharpe'])
df_compare_5Y['Utility'] = pd.to_numeric(df_compare_5Y['Utility'])
df_compare_5Y['Prob'] = pd.to_numeric(df_compare_5Y['Prob'])
df_compare_7Y['Sharpe'] = pd.to_numeric(df_compare_7Y['Sharpe'])
df_compare_7Y['Utility'] = pd.to_numeric(df_compare_7Y['Utility'])
df_compare_7Y['Prob'] = pd.to_numeric(df_compare_7Y['Prob'])

In [24]:
# import pickle
# # Store data (serialize)
# with open('df_compare_5Y_Q2.pickle', 'wb') as handle:
#     pickle.dump(df_compare_5Y, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('df_compare_7Y_Q2.pickle', 'wb') as handle:
#     pickle.dump(df_compare_7Y, handle, protocol=pickle.HIGHEST_PROTOCOL) 
 

In [20]:
df_results = pd.DataFrame(index=['5Y_sharpe','5Y_uti','5Y_prob',\
    '7Y_sharpe','7Y_uti','7Y_prob'],columns=['x1','x2','x3','x4'])

df_results.iloc[0,:] = df_grid.iloc[df_compare_5Y[['Sharpe']].idxmax(),:]
df_results.iloc[1,:] = df_grid.iloc[df_compare_5Y[['Utility']].idxmax(),:]
df_results.iloc[2,:] = df_grid.iloc[df_compare_5Y[['Prob']].idxmax(),:]
df_results.iloc[3,:] = df_grid.iloc[df_compare_7Y[['Sharpe']].idxmax(),:]
df_results.iloc[4,:] = df_grid.iloc[df_compare_7Y[['Utility']].idxmax(),:]
df_results.iloc[5,:] = df_grid.iloc[df_compare_7Y[['Prob']].idxmax(),:]

In [26]:
# with open('ratios_opt_Q2.pickle', 'wb') as handle:
#     pickle.dump(df_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
df_results

,x1,x2,x3,x4
5Y_sharpe,0.23829047253115743,0.09224243267628252,0.16935657900502488,0.6694670947925601
5Y_uti,1.9817901066547705,-0.10873229426158981,1.8403095358621009,-0.8730578123931807
5Y_prob,0.36424434084062973,1.5736828795302582,1.9697562926494006,-0.9379272203708879
7Y_sharpe,0.23829047253115743,0.09224243267628252,0.16935657900502488,0.6694670947925601
7Y_uti,1.9817901066547705,-0.10873229426158981,1.8403095358621009,-0.8730578123931807
7Y_prob,0.40278169387381246,1.463911571241884,0.897755823229168,-0.8666932651156964


In [21]:
# if not allow shorting
df_results

,x1,x2,x3,x4
5Y_sharpe,0.09519399216000024,0.09605673685818439,0.2794119541475075,0.8087492709818154
5Y_uti,0.014886451338823381,0.07979088125463485,1.839701523727646,0.9053226674065418
5Y_prob,0.26435170579256395,0.0901087301934036,1.8779855495504878,0.6455395640140325
7Y_sharpe,0.19412821283008763,0.130897679253146,0.17178930119158897,0.6749741079167664
7Y_uti,0.014886451338823381,0.07979088125463485,1.839701523727646,0.9053226674065418
7Y_prob,0.25221839175625105,0.3891024263529699,0.8797866192512023,0.3586791818907791
